In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import string

In [2]:
imdb_movies = pd.read_csv("IMDb movies.csv")

C:\Users\emirhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_recsys=imdb_movies[['original_title', 'genre', 'director', 'actors', 'description']].reset_index(drop = True)
data_recsys.head()

,original_title,genre,director,actors,description
0,Miss Jerry,Romance,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.
1,The Story of the Kelly Gang,"Biography, Crime, Drama",Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...
2,Den sorte drøm,Drama,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...
3,Cleopatra,"Drama, History",Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...
4,L'Inferno,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...


In [4]:
data_recsys.set_index("original_title", inplace = True)

In [5]:
data_recsys["genre"] = data_recsys["genre"].fillna('').astype('str').str.lower()
data_recsys["genre"] = data_recsys["genre"].str.split(',')

In [6]:
data_recsys["director"] = data_recsys["director"].fillna('').astype("str").str.lower()
data_recsys["director"] = data_recsys["director"].str.split(',')

data_recsys["actors"] = data_recsys["actors"].fillna('').astype("str").str.lower()
data_recsys["actors"] = data_recsys["actors"].str.split(',')

In [7]:
data_recsys.head()

,genre,director,actors,description
original_title,,,,
Miss Jerry,[romance],[alexander black],"[blanche bayliss, william courtenay, chaunce...",The adventures of a female reporter in the 1890s.
The Story of the Kelly Gang,"[biography, crime, drama]",[charles tait],"[elizabeth tait, john tait, norman campbell,...",True story of notorious Australian outlaw Ned ...
Den sorte drøm,[drama],[urban gad],"[asta nielsen, valdemar psilander, gunnar he...",Two men of high rank are both wooing the beaut...
Cleopatra,"[drama, history]",[charles l. gaskill],"[helen gardner, pearl sindelar, miss fieldin...",The fabled queen of Egypt's affair with Roman ...
L'Inferno,"[adventure, drama, fantasy]","[francesco bertolini, adolfo padovan]","[salvatore papa, arturo pirovano, giuseppe d...",Loosely adapted from Dante's Divine Comedy and...


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emirhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
data_recsys['description'] = data_recsys['description'].fillna('').astype('str').str.lower()
data_recsys['description'] = data_recsys['description'].str.translate(str.maketrans('', '', string.punctuation))

listStopwords = set(stopwords.words('english'))
filtered = []
ps = PorterStemmer() 
for i, text in enumerate(data_recsys['description'].str.split()):
    for word in text:
        # Filtering/Removing stopwords in the text
        if word not in listStopwords:
            # Stemming words
            word_stemmed = ps.stem(word)
            filtered.append(word_stemmed)
    data_recsys['description'][i] = filtered
    filtered = []

In [10]:
data_recsys.head()

,genre,director,actors,description
original_title,,,,
Miss Jerry,[romance],[alexander black],"[blanche bayliss, william courtenay, chaunce...","[adventur, femal, report, 1890]"
The Story of the Kelly Gang,"[biography, crime, drama]",[charles tait],"[elizabeth tait, john tait, norman campbell,...","[true, stori, notori, australian, outlaw, ned,..."
Den sorte drøm,[drama],[urban gad],"[asta nielsen, valdemar psilander, gunnar he...","[two, men, high, rank, woo, beauti, famou, equ..."
Cleopatra,"[drama, history]",[charles l. gaskill],"[helen gardner, pearl sindelar, miss fieldin...","[fabl, queen, egypt, affair, roman, gener, mar..."
L'Inferno,"[adventure, drama, fantasy]","[francesco bertolini, adolfo padovan]","[salvatore papa, arturo pirovano, giuseppe d...","[loos, adapt, dant, divin, comedi, inspir, ill..."


In [11]:
data_recsys['bunch_of_words'] = ''
for i, text in data_recsys.iterrows():
    words = ''
    for col in data_recsys.columns:
        words = words + ' '.join(text[col]) + ' '
    data_recsys['bunch_of_words'][i] = words

In [12]:
data_recsys.head()

,genre,director,actors,description,bunch_of_words
original_title,,,,,
Miss Jerry,[romance],[alexander black],"[blanche bayliss, william courtenay, chaunce...","[adventur, femal, report, 1890]",romance alexander black blanche bayliss willi...
The Story of the Kelly Gang,"[biography, crime, drama]",[charles tait],"[elizabeth tait, john tait, norman campbell,...","[true, stori, notori, australian, outlaw, ned,...",biography crime drama charles tait elizabeth...
Den sorte drøm,[drama],[urban gad],"[asta nielsen, valdemar psilander, gunnar he...","[two, men, high, rank, woo, beauti, famou, equ...",drama urban gad asta nielsen valdemar psiland...
Cleopatra,"[drama, history]",[charles l. gaskill],"[helen gardner, pearl sindelar, miss fieldin...","[fabl, queen, egypt, affair, roman, gener, mar...",comedy drama eduardo mignogna norma aleandro ...
L'Inferno,"[adventure, drama, fantasy]","[francesco bertolini, adolfo padovan]","[salvatore papa, arturo pirovano, giuseppe d...","[loos, adapt, dant, divin, comedi, inspir, ill...",adventure drama fantasy francesco bertolini ...


In [13]:
count = CountVectorizer()
count_matrix = count.fit_transform(data_recsys['bunch_of_words']).astype(np.uint8)

In [15]:
del imdb_movies

In [ ]:
# Calculate Cosine Similarity
# Cosine similarity is a metric used to measure how similar the documents are irrespective of their size

chunk_size = 500 
matrix_len = count_matrix.shape[0] # Not sparse numpy.ndarray

# Calculate cosine similarity chunk by chunk
def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=count_matrix[start:end], Y=count_matrix)
cosine_similarity_all = []
i=0
for chunk_start in range(0, matrix_len, chunk_size):
    
    # Initialize first cosine sim chunk (for first concatenating chunks purpose)
    if i == 0: 
        cosine_sim = similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)
    
    # Initialize other cosine sim chunk, then concatenating chunk by chunk untill all chunks concatenated
    else :
        cosine_similarity_chunk= similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)
        # Use type data float32 for reduce memory usage
        cosine_sim = np.concatenate((cosine_sim.astype(np.float32), cosine_similarity_chunk.astype(np.float32)))
    
    # Change value i != 0 for execute else statement, because we dont need execute if statement anymore (if statement only to initialize first chunk for first concatenating purpose)
    i= 1

In [43]:
index_movies = pd.Series(data_recsys.index)

In [55]:
def recommendation_movies(title, cosine_sim = cosine_sim):
    recommended_movies = []
    index_movie_input = index_movies[index_movies == title].index[0]
    score_movies = pd.Series(cosine_sim[index_movie_input]).sort_values(ascending = False)
    top_10_index_movies = list(score_movies.iloc[1:11].index)
    # Get movies title and year by index (top 10 movies)
    for i in top_10_index_movies:
        recommended_movies.append(imdb_movies['original_title'].iloc[i] + ' (' + str(imdb_movies['year'].iloc[i]) + ')')
    return recommended_movies